<a href="https://colab.research.google.com/github/masonnystrom/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 2*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

### Setup

You can work locally (follow the [local setup instructions](https://lambdaschool.github.io/ds/unit2/local/)) or on Colab (run the code cell below).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [5]:
# get val set 
train, val = train_test_split(train, train_size=.80, test_size=.20,
                              stratify=train['status_group'], random_state=2,)
train.shape, val.shape

((47520, 41), (11880, 41))

In [0]:
def wrangle(X):
  """wrangle function for data"""
  X = X.copy()
  # small latitutde values
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # cols with zeros
  cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                     'gps_height', 'population']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0,np.nan)
    X[col+'_MISSING'] = X[col].isnull()
  
  # drop duplicates
  duplicates = ['quantity_group', 'payment_type', 'water_quality', 
                'source_type', 'waterpoint_type_group','extraction_type_group',
                'extraction_type','management']
  X = X.drop(columns=duplicates)

  # drop recorded by(only one group recorded everything) and id(always varies/random)
  unusable_variance = ['recorded_by']
  X = X.drop(columns=unusable_variance)

  #convert to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  # create month, year columns
  X['year_recorded'] = X['date_recorded'].dt.year 
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day 
  X = X.drop(columns='date_recorded')

  #Engineer feature: how many years since the construction
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_MISSING'] = X['years'].isnull()

  # wrangel and return dataframe
  return X 


In [0]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
#Set the features
target = 'status_group'

train_features = train.drop(columns=[target, 'id'])

# numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()

categorical_features = cardinality[cardinality <= 50].index.tolist()

features = numeric_features + categorical_features

# Testing for Kaggle

### Feature engineering

In [0]:
# Get a list of the top funders
top_funders = train['funder'].value_counts()[:30].index
 
train.loc[~train['funder'].isin(top_funders), 'funder'] = 'OTHER'
val.loc[~val['funder'].isin(top_funders), 'funder'] = 'OTHER'
test.loc[~test['funder'].isin(top_funders), 'funder'] = 'OTHER'

In [0]:
top_wpt = train['wpt_name'].value_counts()[:30].index

train.loc[~train['wpt_name'].isin(top_wpt), 'wpt_name'] = 'OTHER'
val.loc[~val['wpt_name'].isin(top_wpt), 'wpt_name'] = 'OTHER'
test.loc[~test['wpt_name'].isin(top_wpt), 'wpt_name'] = 'OTHER'


In [0]:
top_installer = train['installer'].value_counts()[:50].index

train.loc[~train['installer'].isin(top_installer), 'installer'] = 'OTHER'
val.loc[~val['installer'].isin(top_installer), 'installer'] = 'OTHER'
test.loc[~test['installer'].isin(top_installer), 'installer'] = 'OTHER'

In [0]:
top_subvillage = train['subvillage'].value_counts()[:200].index
 
train.loc[~train['subvillage'].isin(top_subvillage), 'subvillage'] = 'OTHER'
val.loc[~val['subvillage'].isin(top_subvillage), 'subvillage'] = 'OTHER'
test.loc[~test['subvillage'].isin(top_subvillage), 'subvillage'] = 'OTHER'

In [0]:
top_ward = train['ward'].value_counts()[:50].index

train.loc[~train['ward'].isin(top_ward), 'ward'] = 'OTHER'
val.loc[~val['ward'].isin(top_ward), 'ward'] = 'OTHER'
test.loc[~test['ward'].isin(top_ward), 'ward'] = 'OTHER'

In [0]:
top_num_private = train['num_private'].value_counts()[:30].index
 
train.loc[~train['num_private'].isin(top_num_private), 'num_private'] = 'OTHER'
val.loc[~val['num_private'].isin(top_num_private), 'num_private'] = 'OTHER'
test.loc[~test['num_private'].isin(top_num_private), 'num_private'] = 'OTHER'

In [0]:
top_scheme_name = train['scheme_name'].value_counts()[:30].index
 
train.loc[~train['scheme_name'].isin(top_scheme_name), 'scheme_name'] = 'OTHER'
val.loc[~val['scheme_name'].isin(top_scheme_name), 'scheme_name'] = 'OTHER'
test.loc[~test['scheme_name'].isin(top_scheme_name), 'scheme_name'] = 'OTHER'

In [0]:
top_lga = train['lga'].value_counts()[:50].index
 
train.loc[~train['lga'].isin(top_lga), 'lga'] = 'OTHER'
val.loc[~val['lga'].isin(top_lga), 'lga'] = 'OTHER'
test.loc[~test['lga'].isin(top_lga), 'lga'] = 'OTHER'

In [0]:
target = 'status_group'
X_train = train.drop(columns= [target, 'id','construction_year'])
y_train = train[target]

X_val = val.drop(columns= [target, 'id','construction_year'])
y_val = val[target]

X_test = test.drop(columns=['id','construction_year'])

In [0]:
#imports
import category_encoders as ce
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# RF Classifier Model

In [31]:
# RF Classifier
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    StandardScaler(), 
    RandomForestClassifier(n_estimators=128, max_features="auto", 
                           random_state=2, n_jobs=-1, warm_start=False,
                           min_samples_leaf=3)
)
pipeline.fit(X_train, y_train)
print('Train Accuracy:', pipeline.score(X_train, y_train))
print('Validation Accuracy:', pipeline.score(X_val, y_val))

Train Accuracy: 0.8941077441077441
Validation Accuracy: 0.813047138047138


# Regression Model

In [0]:
# encode status_group as an integer in funcitonal
train['functional'] = (train['status_group']=='functional').astype(int)
val['functional'] = (val['status_group']=='functional').astype(int)
test['functional'] = (val['status_group']=='functional').astype(int)

In [0]:
target = 'functional'
X_train = train.drop(columns= [target,'id','construction_year','status_group'])
y_train = train[target]

X_val = val.drop(columns= [target,'id','construction_year','status_group'])
y_val = val[target]

X_test = test.drop(columns=[target,'id','construction_year'])

In [53]:
# RF Regression
pipeline = make_pipeline(
    ce.TargetEncoder(min_samples_leaf=4, smoothing=1), 
    SimpleImputer(strategy='mean'),
    RandomForestRegressor(n_estimators=128, max_features="auto", 
                           random_state=2, n_jobs=-1, warm_start=False,
                           min_samples_leaf=3) 
)
pipeline.fit(X_train, y_train)
print('Train Accuracy:', pipeline.score(X_train, y_train))
print('Validation Accuracy:', pipeline.score(X_val, y_val))

Train Accuracy: 0.7982406484244337
Validation Accuracy: 0.5090624560726529


In [0]:
from sklearn.model_selection import cross_val_score

# Search and Fitting Hyperparameters

In [62]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_distributions = {
    'targetencoder__min_samples_leaf': randint(1, 1000), 
    
#     Remove this hyperparameter, because of an issue: 
#     https://github.com/scikit-learn-contrib/categorical-encoding/issues/184
#     'targetencoder__smoothing': uniform(1, 1000), 
    
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.8s


KeyboardInterrupt: ignored

In [58]:
pipeline = search.best_estimator_

NameError: ignored

# Submission 

In [0]:
y_pred = pipeline.predict(X_test) 

# make submission
submission = test[['id']].copy()
submission['status_group'] = y_pred

In [0]:
submission.to_csv('waterpump-submission-07.csv', index=False)
sub_df = pd.read_csv('/content/waterpump-submission-07.csv')
sub_df.shape

# Random Forrest Important Features

In [0]:
# get importances
%matplotlib inline
import matplotlib.pyplot as plt

# get encoded values
encoder = pipeline.named_steps['ordinalencoder']
encoded = encoder.transform(X_train)

# get importances
rf = pipeline.named_steps['randomforestclassifier']
importances = pd.Series(rf.feature_importances_, encoded.columns)

In [0]:
n = 50
plt.figure(figsize=(10, n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh();